In [4]:
#import sys
#!conda install --yes --prefix {sys.prefix} xgboost

In [5]:
import pandas as pd
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
#from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import xgboost
import random
import matplotlib
from matplotlib import pyplot as plt

%matplotlib inline

## Pull the tumor gene expression data from kids first

In [6]:
gene_expression_data = pd.read_table("/sbgenomics/project-files/tumor-gene-expression-rsem-tpm-collapsed.tsv").transpose()

In [7]:
gene_expression_data.head()

,0,1,2,3,4,5,6,7,8,9,...,36708,36709,36710,36711,36712,36713,36714,36715,36716,36717
gene_id,MT-CO3,MT-CO2,MT-ATP6,MT-ND4,MT-CO1,MT-RNR2,MT-CYB,MT-ND2,MT-ND1,MT-ATP8,...,CT45A6,CT45A8,MIR718,RNU6-109P,RNU1-97P,TRAPPC2P9,TTTY17B,TRAPPC2P5,USP9YP9,RNU1-40P
BS_DERH44Z2,7458.52,8288.59,6606.88,6398.61,10211.13,2.04,3267.5,5137.5,4806.1,11633.16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BS_CXGWN1W5,3883.93,3047.39,2996.46,4019.43,5824.88,2.51,1691.26,2157.91,3666.67,5305.92,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BS_QS2NKZW3,4849.37,3168.87,3003.09,4011.17,6941.22,23.16,2616.85,3322.97,5265.57,3805.77,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BS_135GYQQD,1335.7,1378.69,932.8,1572.16,2357.34,0.48,687.46,846.49,965.47,1048.87,...,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
genes = gene_expression_data.iloc[0,:]

In [9]:
gene_expression_data = gene_expression_data.iloc[1:,:]
gene_expression_data.columns = genes

In [10]:
gene_expression_data.head()

gene_id,MT-CO3,MT-CO2,MT-ATP6,MT-ND4,MT-CO1,MT-RNR2,MT-CYB,MT-ND2,MT-ND1,MT-ATP8,...,CT45A6,CT45A8,MIR718,RNU6-109P,RNU1-97P,TRAPPC2P9,TTTY17B,TRAPPC2P5,USP9YP9,RNU1-40P
BS_DERH44Z2,7458.52,8288.59,6606.88,6398.61,10211.13,2.04,3267.5,5137.5,4806.1,11633.16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BS_CXGWN1W5,3883.93,3047.39,2996.46,4019.43,5824.88,2.51,1691.26,2157.91,3666.67,5305.92,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BS_QS2NKZW3,4849.37,3168.87,3003.09,4011.17,6941.22,23.16,2616.85,3322.97,5265.57,3805.77,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BS_135GYQQD,1335.7,1378.69,932.8,1572.16,2357.34,0.48,687.46,846.49,965.47,1048.87,...,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BS_HBMHZEGZ,2647.82,2342.92,2144.0,3229.79,4461.07,1.39,1638.03,2052.57,2867.3,3588.04,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
gene_expression_data = gene_expression_data.astype(float)

In [12]:
gene_expression_data.shape[0]

1284

## Import gene-cluster assignments from pvclust

In [12]:
cluster_assignments = pd.read_csv("/sbgenomics/project-files/cluster_assignments.csv")

In [17]:
clustered_patients = gene_expression_data.loc[cluster_assignments['biospecimen'],:]

In [19]:
clustered_patients.head()

gene_id,MT-CO3,MT-CO2,MT-ATP6,MT-ND4,MT-CO1,MT-RNR2,MT-CYB,MT-ND2,MT-ND1,MT-ATP8,...,CT45A6,CT45A8,MIR718,RNU6-109P,RNU1-97P,TRAPPC2P9,TTTY17B,TRAPPC2P5,USP9YP9,RNU1-40P
BS_TGDZ491N,1740.82,1060.56,1009.86,1250.39,2592.68,0.78,816.91,856.54,1050.92,1828.90,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BS_FXDGYFBR,2518.75,1551.76,1162.41,1937.14,3599.38,0.64,804.84,843.01,1396.26,1113.39,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BS_DERH44Z2,7458.52,8288.59,6606.88,6398.61,10211.13,2.04,3267.50,5137.50,4806.10,11633.16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BS_CXGWN1W5,3883.93,3047.39,2996.46,4019.43,5824.88,2.51,1691.26,2157.91,3666.67,5305.92,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BS_QS2NKZW3,4849.37,3168.87,3003.09,4011.17,6941.22,23.16,2616.85,3322.97,5265.57,3805.77,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
all(clustered_patients.index == cluster_assignments['biospecimen'])

True

In [24]:
cluster_assignments['cluster']

0     1
1     1
2     2
3     2
4     2
     ..
95    2
96    2
97    2
98    2
99    2
Name: cluster, Length: 100, dtype: int64

In [35]:
xtrain, xtest, ytrain, ytest = train_test_split(clustered_patients, cluster_assignments['cluster'], test_size=0.20)

In [41]:
set(ytrain)

{1, 2}

In [42]:
clf = OneVsRestClassifier(xgboost.XGBClassifier())
clf.fit(clustered_patients,cluster_assignments['cluster'])

/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:41:38] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            enable_categorical=False,
                                            gamma=None, gpu_id=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None,
                                            max_delta_step=None, max_depth=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
     

In [44]:
clf.estimators_

[XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
               gamma=0, gpu_id=-1, importance_type=None,
               interaction_constraints='', learning_rate=0.300000012,
               max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
               monotone_constraints='()', n_estimators=100, n_jobs=8,
               num_parallel_tree=1, predictor='auto', random_state=0,
               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
               tree_method='exact', validate_parameters=1, verbosity=None)]

In [47]:
feature_importance = pd.DataFrame(clf.estimators_[0].feature_importances_) #pd.DataFrame({'Cluster_{0}'.format(i):clf.estimators_[i].feature_importances_ for i in range(len(set(cluster_assignments['cluster'])))})

In [48]:
feature_importance.index = genes

In [60]:
#feature_importance[feature_importance['Cluster_0']!=0.0]
important_genes = feature_importance[feature_importance[0]!=0.0].index

In [65]:
pd.DataFrame({'Cluster':0,'Genes':important_genes}).to_csv("/sbgenomics/output-files/cluster_important_genes.csv",index=False)